In [ ]:
import os, sys, time
import tarfile
import pickle as pkl

In [ ]:
data_dir = '../data'
msd_dir = os.path.join(data_dir, 'msd/')
fmap  = os.path.join(data_dir, 'aotm-2011/map_song_track.pkl')

## Data loading

Load `song_id` --> `track_id` mapping: a song may correspond to multiple tracks.

In [ ]:
song2TrackID = pkl.load(open(fmap, 'rb'))

In [ ]:
{ k : song2TrackID[k] for k in sorted(song2TrackID.keys())[:10] }

## Extract all related track files

In [ ]:
trackIDs = sorted({trackID for value in song2TrackID.values() for trackID in value})

In [ ]:
len(trackIDs)

In [ ]:
trackIDs[:10]

In [ ]:
tar = None
flag = None
cnt = 0
for trackID in trackIDs:
    cnt += 1
    sys.stdout.write("\r%d / %d" % (cnt, len(trackIDs)))
    sys.stdout.flush()
    
    ftrack = trackID[2] + '/' + trackID[3] + '/' + trackID[4] + '/' + trackID + '.h5'
    
    # practical solution: generate bash commands
    # workflow: extract .tar.gz -> move files-of-interest -> remove all extracted files; repeat this.
    #fnew = 'files/' + trackID + '.h5'
    #if os.path.exists(os.path.join(data_dir, fnew)): continue
    #print('mv', ftrack, fnew)
    
    fnew = os.path.join(data_dir, 'files/' + trackID + '.h5')
    ftar = os.path.join(msd_dir, trackID[2] + '.tar.gz')
    
    if os.path.exists(fnew): continue
    
    if flag is None or flag != trackID[2]:
        flag = trackID[2]
        if tar is not None:
            tar.close()    
        tar = tarfile.open(name=ftar, mode='r:gz')
        
    if tar is None:
        tar = tarfile.open(name=ftar, mode='r:gz')
    
    fdr = tar.extractfile(ftrack)
    with open(fnew, 'wb') as fdw:
        fdw.write(fdr.read())
    fdr.close()